In [ ]:
p piinstall --upgrade webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# 設定 Chrome 瀏覽器選項
chrome_options = Options()
chrome_options.add_argument("--headless")  # 如果不需要顯示瀏覽器，可以啟用這行
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36')

chrome_options.binary_location ='C:\Program Files\Google\Chrome\Application\chrome.exe'
# 使用 Service 配置並啟動 ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 訪問網頁
keyword = '家電'
url = 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12694615&Area=search&mdiv=403&oid=1_8&cid=index&kw=%E5%AE%B6%E9%9B%BB'
driver.get(url)


# 等待並點擊商品評價按鈕
review_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "ul.vendordetailmenu li.goodsCommendLi span"))
)
# print("按鈕已經找到，可以點擊")
review_button.click()  # 點擊商品評價按鈕

# 假設這是頁面上的分頁區域
page_number_element = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.pageArea span'))
)
# 找到最後一個頁面按鈕
total_pages = page_number_element[1].text[-2:]
# print(f"總共的頁數是: {total_pages}")

# 開始抓取評論資料
comments_data = []
ratings_data = []

# 等待加載完成
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.CommentContainer'))
)

# 抓取第一頁評論
comments = driver.find_elements(By.CSS_SELECTOR, 'p.Comment')
for comment in comments:
    comment_text = comment.text
    comments_data.append(comment_text)
    # print(comment.text)  # 或是其他處理評論的邏輯

# 等待加載完成
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.InfoContainer'))
)

rating_star_group = driver.find_elements(By.CSS_SELECTOR, 'div.RatingStarGroup')
for i in range(len(rating_star_group)):
    score = rating_star_group[i].get_attribute('score')
    ratings_data.append(score)
    # print(f'score: {score}')

# 抓取二到最後一頁評論
for time in range(2, int(total_pages)+1):
    next_page_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f'div.pageArea dd[pageidx="{str(time)}"] a'))
    )
    # print(f"第{time}頁按鈕已經找到，可以點擊")
    next_page_button.click()

    # 等待加載評論完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.reviewCard'))
    )

    # 抓取評論
    comments = driver.find_elements(By.CSS_SELECTOR, 'div.reviewCardInner div.CommentContainer:not(.ReplyContainer .CommentContainer) p.Comment')
    for comment in comments:
        comment_text = comment.text
        comments_data.append(comment_text)
        # print(comment.text)  # 或是其他處理評論的邏輯

    # 等待加載評分完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.InfoContainer'))
    )

    # 抓取評分
    rating_star_group = driver.find_elements(By.CSS_SELECTOR, 'div.RatingStarGroup')
    for i in range(len(rating_star_group)):
        score = rating_star_group[i].get_attribute('score')
        ratings_data.append(score)
        # print(f'score: {score}')

comments_data = [[comment] for comment in comments_data]
ratings_data = [[rating] for rating in ratings_data]
# # print(comments_data)


# 關閉瀏覽器
driver.quit()


WebDriverException: Message: unknown error: no chrome binary at C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
#0 0x594e5e8174e3 <unknown>
#1 0x594e5e546c76 <unknown>
#2 0x594e5e56d5e0 <unknown>
#3 0x594e5e56c029 <unknown>
#4 0x594e5e5aaccc <unknown>
#5 0x594e5e5aa47f <unknown>
#6 0x594e5e5a1de3 <unknown>
#7 0x594e5e5772dd <unknown>
#8 0x594e5e57834e <unknown>
#9 0x594e5e7d73e4 <unknown>
#10 0x594e5e7db3d7 <unknown>
#11 0x594e5e7e5b20 <unknown>
#12 0x594e5e7dc023 <unknown>
#13 0x594e5e7aa1aa <unknown>
#14 0x594e5e8006b8 <unknown>
#15 0x594e5e800847 <unknown>
#16 0x594e5e810243 <unknown>
#17 0x79974ad09ac3 <unknown>


In [ ]:
len(comments_data)

488

In [ ]:
comments_data

[['商品正確、出貨快速、謝謝'],
 ['很棒棒棒棒棒棒棒棒棒棒棒'],
 ['商品出貨速度快，包裝完整'],
 ['氣炸鍋贈送的盤子三個，我還沒收到請你們立刻配送給我謝謝'],
 ['出貨速度快，商品無誤，包裝完整！下次有需要會再繼續回購，謝謝！'],
 ['看起來還不錯，就是還没開封'],
 ['商品的品質優，價錢也合理，重點是過年前有辦法兩天就到貨，實在太棒！'],
 ['毛邊屑屑很多'],
 ['外觀高雅，品牌有保證，希望能耐用'],
 ['不錯'],
 ['商品正確、出貨快速、謝謝'],
 ['很大一台 容量很足夠 可以炸很多東西'],
 ['尾牙送禮大方又漂亮！'],
 ['推拉卡卡的，以女生力氣無法順利使用 已申請換貨，希望可以拿到品質優良的'],
 ['價格合理實惠希望好用'],
 ['好用！氣炸任何食物都很棒！ 缺點就是，放進去的拉盒最後一下要用力一點，不然會放不進去完全！ 不然整體都很棒！連聲音都很安靜！'],
 ['比想像中的大台，還未開始使用，商品蠻有質感的'],
 ['符合期待，顏色喜歡，趁特價買起來。'],
 ['出貨速度非常快速產品很優'],
 ['買來送給客戶年終摸彩用很值得'],
 ['外觀好看，但就是推進去的時候 有點卡卡的'],
 ['雖然比想像中大台，但很好用，第一次使用氣炸鍋稍微看一下說明就可以上手。'],
 ['目前使用其實還不錯 剛開始有一些塑膠味很重！'],
 ['出貨快'],
 ['品質優良，商品好用，送貨快速！'],
 ['便宜好用，空間又大，黑色高貴'],
 ['【Electrolux 伊萊克斯】5公升觸控式氣炸鍋(E6AF1-520K), 人性化觸控面板設計,8種預設程式一鍵輕鬆完成,安全斷電防燙設置momo購物網總是優惠便宜好價格,值得推薦！'],
 ['操作簡單 介面清晰明瞭'],
 ['快速的出貨速度'],
 ['好用夠大'],
 ['👍'],
 ['用後回饋：外鍋與外殼的銜接處是塑膠？每次使用(180℃)都是那個味道，橘子皮開鍋兩次沒效，可能要用鳳梨皮?!'],
 ['出貨速度算快 商品也包裝完整'],
 ['品質優良，送貨快速。'],
 ['品質優良，送貨快速。'],
 ['大品牌就是好質感！'],
 ['看網路評價不錯 還沒開機 先買起來'],
 ['因為家裡氣炸鍋壞了 找人修 花了500元 仍然

In [ ]:
len(ratings_data)

488

In [ ]:
# 儲存評論到 CSV 文件
with open("comments.csv", mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["Comment"]) # 寫入標題列
    writer.writerows(comments_data) # 寫入評論資料